## ragas评估

### rag pipline

In [1]:
import warnings
warnings.filterwarnings('ignore')
from model import RagEmbedding, RagLLM, QwenLLM
from langchain_chroma import Chroma
import chromadb

In [2]:
llm = RagLLM()

In [3]:
prompt_template = """
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接回答[不知道,请咨询HR] \
问题：{question} 
"{context}"
回答：
"""

In [4]:
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [6]:
embedding_model = RagEmbedding()

In [7]:
zhidu_db = Chroma("zhidu_db", 
                  embedding_model.get_embedding_fun(), 
                  client=chroma_client)

In [11]:
def run_rag_pipline_without_stream(query, k=3):
    related_docs = zhidu_db.similarity_search(query, k=k)
    context_list = [f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(related_docs)]
    context = "\n".join(context_list)

    llm_prompt = prompt_template.replace("{question}", query).replace("{context}", context)
    response = llm(llm_prompt, stream=False)
    return response, context_list

### 构建评估数据集
- 问题
- 标准答案
- 上下文信息
- 生成的答案

In [9]:
questions = [
    "伙食补助费标准是什么?",
    "出差可以买意外保险吗？需要自己购买吗",
]
ground_truths = [
    "伙食补助费标准: 西藏、青海、新疆 120元/人、天 其他省份 100元/人、天",
    "出差可以购买交通意外保险，由单位统一购买，不再重复购买",
]

In [13]:
answers = []
contexts = []

for query in questions:
    response, context_list = run_rag_pipline_without_stream(query, k=3)
    answers.append(response)
    contexts.append(context_list)


In [14]:
from datasets import Dataset

In [15]:
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

In [16]:
dataset = Dataset.from_dict(data)

In [17]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [18]:
from ragas import evaluate
from ragas import RunConfig

In [19]:
eval_llm = QwenLLM()
embedding_model = RagEmbedding()
eval_embedding_fn = embedding_model.get_embedding_fun()

In [20]:
config = RunConfig(timeout=1200, log_tenacity=True) 

In [21]:
result = evaluate(
    dataset = dataset,
    llm=eval_llm,
    embeddings=eval_embedding_fn,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    raise_exceptions=True,
    run_config=config
)

df = result.to_pandas()

Evaluating: 100%|██████████| 8/8 [02:47<00:00, 20.97s/it]


In [22]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,伙食补助费标准是什么?,伙食补助费的标准为：西藏、青海、新疆地区120元/人、天；其他省份100元/人、天。,[上下文1: <table><caption>伙食补助费参考以下标准：</caption>\...,伙食补助费标准: 西藏、青海、新疆 120元/人、天 其他省份 100元/人、天,1.0,1.0,1.0,0.988139
1,出差可以买意外保险吗？需要自己购买吗,可以购买意外保险，每人次可以购买交通意外保险一份。如果公司已经统一购买了交通意外保险，则不需...,[上下文1: 享受副部级待遇及以上人员出差，因工作需要，随行一人可乘坐同等级交通工具；乘坐飞...,出差可以购买交通意外保险，由单位统一购买，不再重复购买,1.0,1.0,0.8,0.763618
